In [128]:
# import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
# %matplotlib inline

In [129]:
import datetime
import cPickle as pickle
import csv
import numpy as np
import random
import sys
from file_util import create_folder
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [130]:
# name_dataset        = 'real_world_articles/old'
name_dataset        = 'real_world_articles/new'
path_raw_data       = '../data/raw/' + name_dataset + '/'
path_processed_data = '../data/' + name_dataset + '/whole/'

In [131]:
create_folder(path_raw_data)
# create_folder(path_processed_data + '/train')
# create_folder(path_processed_data + '/dev')
create_folder(path_processed_data + '/test')
# create_folder(path_processed_data + '/debug')

## read voca from dic

In [132]:
target_dic = '../data/news-19_paragraph_swap-random-1m/whole/dic_mincutN.txt'

list_voca = []
with open(target_dic, 'r') as f:
    list_voca = f.readlines()
    list_voca = [x.strip() for x in list_voca]

dic_voca = {}
for voca in list_voca:
    dic_voca[voca] = len(dic_voca)

len(dic_voca)

196779

## for data processing

In [133]:
import copy
dic_voca_lower = copy.deepcopy(dic_voca)

In [134]:
dic_voca_lower['<eos>'] = dic_voca_lower['<EOS>']
dic_voca_lower['<eop>'] = dic_voca_lower['<EOP>']

del dic_voca_lower['<EOS>']
del dic_voca_lower['<EOP>']

In [135]:
len(dic_voca_lower)

196779

In [136]:
print(dic_voca_lower[''], dic_voca_lower['<UNK>'], dic_voca_lower['<eos>'], dic_voca_lower['<eop>'])

(0, 1, 2, 3)


## stats

In [137]:
import csv
import sys
import numpy as np

data= []
with open(path_raw_data + 'test.tsv', 'r') as f:
    data_csv = csv.reader(f, delimiter='\t',
                          quoting=csv.QUOTE_NONE)
    for row in data_csv:
        data.append(row)

In [138]:
def print_info(data):
    print("mean", np.average(data))
    print("std", np.std(data))
    print("max", np.max(data))
    print("95.xx coverage", np.average(data) +  2*np.std(data) )
    print("99.73 coverage", np.average(data) +  3*np.std(data) )
    print("99.95 coverage", np.average(data) +  3.5*np.std(data) )
    print("99.99 coverage", np.average(data) +  4*np.std(data) )

In [139]:
head = [x[1].strip() for x in data]
head_len = [len(x.split()) for x in head]
print('head_len')
print_info(head_len)

head_len
('mean', 11.237227486953639)
('std', 3.541768671679837)
('max', 38)
('95.xx coverage', 18.320764830313312)
('99.73 coverage', 21.86253350199315)
('99.95 coverage', 23.63341783783307)
('99.99 coverage', 25.40430217367299)


In [140]:
body = [x[2].strip() for x in data]
body_len = [len(x.split()) for x in body ]
print('body_len')
print_info(body_len)

body_len
('mean', 752.7194849987923)
('std', 392.8987616809875)
('max', 6306)
('95.xx coverage', 1538.5170083607672)
('99.73 coverage', 1931.4157700417547)
('99.95 coverage', 2127.8651508822486)
('99.99 coverage', 2324.3145317227422)


In [141]:
context_len = [len(x.split('<EOP>')) for x in body]
print('context_len')
print_info(context_len)

context_len
('mean', 26.560091900302776)
('std', 12.933705983369837)
('max', 51)
('95.xx coverage', 52.42750386704245)
('99.73 coverage', 65.3612098504123)
('99.95 coverage', 71.82806284209721)
('99.99 coverage', 78.29491583378213)


In [142]:
body_sentence = []
for sent in body:
    sent = sent.split('<EOP>')
    body_sentence.extend(sent)
body_len = [ len(x.split()) for x in body_sentence ]    
print('body_sentence')
print_info(body_len)

body_sentence
('mean', 27.37789446768443)
('std', 18.148781561876078)
('max', 1578)
('95.xx coverage', 63.67545759143658)
('99.73 coverage', 81.82423915331266)
('99.95 coverage', 90.8986299342507)
('99.99 coverage', 99.97302071518874)


# encode to numpy

In [143]:
def fit_length(data, max_len_t, max_len_b):
    data_t, data_b = data
    
    list_zeros = np.zeros(max_len_b, 'int32').tolist()
    fl_data_t = []
    for datum in data_t:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_t:
            fl_data_t.append( datum[:max_len_t] )
        else:
            fl_data_t.append( datum + list_zeros[:(max_len_t-_len)] )
            
    fl_data_b = []
    for datum in data_b:
        try:
            datum = list(datum)
        except:
            pass
        _len = len(datum)
        if _len >= max_len_b:
            fl_data_b.append( datum[:max_len_b] )
        else:
            fl_data_b.append( datum + list_zeros[:(max_len_b-_len)] )
    
    np_data_t = np.asarray(fl_data_t, dtype='int32')
    np_data_b = np.asarray(fl_data_b, dtype='int32')
    
    data = [np_data_t, np_data_b]
    return data

In [144]:
csv_reader = csv.reader(open(path_raw_data + 'test.tsv', 'r'),
                        delimiter='\t',
                        quoting=csv.QUOTE_NONE)

print datetime.datetime.now().isoformat()
ids = []
heads = []
bodys = []
labels = []
for n, row in enumerate(csv_reader):
    
#     if n <  3000000:
#         continue

#     if n >=  3000000:
#         continue
        
                
    if (n+1) % 10000 == 0: print n+1,
    
    ids.append(row[0])
    labels.append(0)
    
    head = []
    for tkn in row[1].lower().strip().split():
        if tkn in dic_voca_lower:
            head.append(dic_voca_lower[tkn])
        else:
            head.append(1)
            
    heads.append(head)
    
    body = []
    for tkn in row[2].lower().strip().split():
        if tkn in dic_voca_lower:
            body.append(dic_voca_lower[tkn])
        else:
            body.append(1)
            
    bodys.append(body)
    
print n+1, 'Done'
print datetime.datetime.now().isoformat() # ~5 mins

2020-05-24T09:41:26.397402
10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 178019 Done
2020-05-24T09:42:03.056171


In [145]:
print datetime.datetime.now().isoformat()
[np_heads, np_bodys] = fit_length([heads, bodys], 25, 2100)
print datetime.datetime.now().isoformat() # ~3 mins

2020-05-24T09:42:03.064060
2020-05-24T09:42:28.684434


In [146]:
print datetime.datetime.now().isoformat()
t_trainpath = path_processed_data + '/test/test_title.npy'
np.save(t_trainpath, np_heads)
b_trainpath = path_processed_data + '/test/test_body.npy'
np.save(b_trainpath, np_bodys)
l_trainpath = path_processed_data + '/test/test_label.npy'
np.save(l_trainpath, labels)
print datetime.datetime.now().isoformat()

2020-05-24T09:42:28.694558
2020-05-24T09:42:29.311515


In [148]:
with open('../data/real_world_articles/new/whole/test/test_label.npy', 'r') as f:
    tt = np.load(f)
tt.shape

(178019,)